In [ ]:
import pandas as pd

In [ ]:
kto_201901 = pd.read_excel('./files/kto_201901.xlsx', header=1, usecols='A:G', skipfooter=4)
kto_201901.head()

In [ ]:
kto_201901.info()

In [ ]:
kto_201901.describe()

In [ ]:
#조건정의

condition = (kto_201901['관광']==0)|(kto_201901['상용']==0)|(kto_201901['공용']==0)|(kto_201901['유학/연수']==0)
kto_201901[condition]

In [ ]:
#칼럼추가

kto_201901['기준년월'] = '2019-01'
kto_201901.head()

In [ ]:
kto_201901['국적'].unique()

In [ ]:
continents_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']
continents_list

In [ ]:
#대륙 정보 제외하고 나라 이름만 남기기

condition = (kto_201901.국적.isin(continents_list) == False)
kto_201901_country = kto_201901[condition]
kto_201901_country['국적'].unique()

In [ ]:
kto_201901_country.head()

In [ ]:
kto_201901_country_newindex = kto_201901_country.reset_index(drop=True)
kto_201901_country_newindex.head()

In [ ]:
continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['오세아니아']*3 + ['아프리카']*2 +['기타대륙'] + ['교포']
print(continents)

In [ ]:
kto_201901_country_newindex['대륙'] = continents
kto_201901_country_newindex.tail(10)

In [ ]:
kto_201901_country_newindex['관광객비율(%)']=round(kto_201901_country_newindex['관광'] / kto_201901_country_newindex['계']*100, 1)
kto_201901_country_newindex.head()

In [ ]:
kto_201901_country_newindex.sort_values(by='관광객비율(%)').head()

In [ ]:
#피벗테이블 만들기

kto_201901_country_newindex.pivot_table(values='관광객비율(%)', index = '대륙', aggfunc = 'mean')

In [ ]:
tourist_sum = sum(kto_201901['관광'])
kto_201901_country_newindex['전체비율(%)'] = \
    round(kto_201901_country_newindex['관광'] / \
    tourist_sum*100, 1)
kto_201901_country_newindex.head()

In [ ]:
kto_201901_country_newindex.sort_values(by='전체비율(%)', ascending=False).head(10)

In [ ]:
def create_kto_data(yy, mm):
    file_path = './files/kto_{}{}.xlsx'.format(yy,mm)
    df = pd.read_excel(file_path, \
                           header=1, usecols='A:G',\
                           skipfooter=4)
    df['기준년월'] = '{}-{}'.format(yy,mm)
    ignore_list = ['아시아주','미주','구주','대양주',\
                   '아프리카주','기타대륙','교포소계']
    condition = (df['국적'].isin(ignore_list)==False)
    df_country = df[condition].reset_index(drop=True)
    continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 \
    + ['오세아니아']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']
    df_country['대륙'] = continents
    df_country['관광객비율(%)'] = round(df_country['관광']/df_country['계'] * 100, 1)
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round(df_country['관광']/tourist_sum * 100, 1)
    return(df_country)

In [ ]:
kto_test = create_kto_data(2018,12)
kto_test.head()

In [ ]:
for yy in range(2010, 2021):
    for mm in range(1, 13):
        ##zfill함수 → mm은 무조건 2칸 채워서
        mm_str = str(mm).zfill(2)
        yymm = '{}{}'.format(yy,mm_str)
        print(yymm)

In [ ]:
df = pd.DataFrame()

In [ ]:
for yy in range(2010, 2021):
    for mm in range(1, 13):
        #try구문
        try:
            temp = create_kto_data(str(yy), str(mm).zfill(2))
            df = df.append(temp, ignore_index=True)
        except:
            pass
df.info()

In [ ]:
df.head()

In [ ]:
df.to_excel('./files/kto_total_20230331.xlsx', index=False)

In [ ]:
from matplotlib import font_manager, rc
import platform 

if platform.system() == 'Windows': 
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else: 
    print('Check your OS system')

In [ ]:
#시계열 그래프

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df['기준년월'], df['관광'])
plt.show()

In [ ]:
#중국 데이터만 추출

condition = (df['국적'].isin(['중국']) == True)
df_filter = df[condition]
df_filter.head()

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df_filter['기준년월'], df_filter['관광'])
plt.title('중국 국적의 관광객 추이')
plt.xlabel('기준년월')
plt.ylabel('관광객수')
plt.xticks(['2010-01', '2011-01', '2012-01', '2013-01', '2014-01', '2015-01', '2016-01', '2017-01', '2018-01', '2019-01', '2020-01'])
plt.show()

In [ ]:
cntry_list = ['중국', '일본', '대만', '미국', '홍콩']
for cntry in cntry_list:
    condition = (df['국적']==cntry)
    df_filter = df[condition]
    plt.figure(figsize=(12,4))
    plt.plot(df_filter['기준년월'], df_filter['관광'])
    plt.title('{} 국적의 관광객 추이'.format(cntry))
    plt.xlabel('기준년월')
    plt.ylabel('관광객수')
    plt.xticks(['2010-01', '2011-01', '2012-01', '2013-01', '2014-01', '2015-01', '2016-01', '2017-01', '2018-01', '2019-01', '2020-01'])
    plt.show()

In [ ]:
#히트맵 그래프

In [ ]:
df['년도'] = df['기준년월'].str.slice(0,4)
df['월'] = df['기준년월'].str.slice(5,7)
df.head()

In [ ]:
condition = (df['국적']=='중국')
df_filter = df[condition]
df_filter.head()

In [ ]:
df_pivot = df_filter.pivot_table(values='관광', index='년도', columns='월')
df_pivot

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_pivot, annot=True, fmt='.0f', cmap='rocket_r')
plt.title('중국 관광객 히트맵')
plt.show

In [ ]:
cntry_list

In [ ]:
for cntry in cntry_list:
    condition = (df['국적']==cntry)
    df_filter = df[condition]
    df_pivot = df_filter.pivot_table(values='관광', index='년도', columns='월')
    plt.figure(figsize=(16,10))
    sns.heatmap(df_pivot, annot=True, fmt='.0f', cmap='rocket_r')
    plt.title('{} 관광객 히트맵'.format(cntry))
    plt.show